# Emulador de uma Máquina de Von Neumann

Projeto desenvolvido para a disciplina de Sistemas de Programação da Escola Politécnica da USP.

## Compilação

O projeto foi desenvolvido no Ubuntu 19.10, e deve funcionar para qualquer sistema Linux. Basta executar o comando `make`
para compilar os arquivos.

## Arquitetura da Máquina de Von Neumann

A Máquina de Von Neumann (MVN) implementada possui instruções de 16 *bits*, sendo os quatro primeiros reservados para o código
de operação (opcode) e os restantes para operando da instrução. Possui apenas dois registradores, o contador de instruções
(PC) e o acumulador (AC). O endereçamento de memória utiliza 12 *bits*, de forma que são disponíveis 4096 posições de memória
acessíveis. Cada posição possui um *byte* de largura.

As operações disponíveis são:

| opcode | mnemônico | efeito |
|--------|----------|--------|
| 0xxx | JP  | PC = xxx
| 1xxx | JZ  | if (AC == 0) PC = xxx
| 2xxx | JNE | if (AC < 0) PC = xxx
| 3xxx | LV  | AC = 0xx (último byte)
| 4xxx | ADD | AC += mem[xxx]
| 5xxx | SUB | AC -= mem[xxx]
| 6xxx | MUL | AC *= mem[xxx]
| 7xxx | DIV | AC /= mem[xxx]
| 8xxx | LD  | AC = mem[xxx]
| 9xxx | MM  | mem[xxx] = AC
| Axxx | SC  | mem[xxx, xxx+1] = PC; PC = xxx+2
| Bxxx | RS  | PC = xxx
| Cxxx | HM  | para a execução
| Dxxx | GD  | lê um byte do arquivo de entrada para AC
| Exxx | PD  | escreve AC no arquivo de saída
| Fxxx | OS  | chamada de sistema (ainda não implementada)

A parte de *input/output* é feita por meio de arquivos em *plain text* dentro da pasta `/filesystem`. Por *default*, o arquivo
de entrada é o `input.txt` e o de saída é o `output.txt`.

### O Motor de Eventos

A MVN foi implementada utilizando o conceito de um motor de eventos. Isto é, em um loop de simulação, a MVN busca uma instrução na memória, decodifica sua operação e operandos e ativa os procedimentos específicos de reação para a simulação das intstruções correspondentes. Ao final de um ciclo, atualiza o ponteiro (*program counter*) para o endereço da prócima instrução a ser executada.

## Utilização

A interface com o usuário é feita por meio de uma interface de linha de comando (CLI):
    

<div style="background:black">
<code style="background:black;color:white">
__     __          _   _                                         __     ____  __ 
\ \   / /__  _ __ | \ | | ___ _   _ _ __ ___   __ _ _ __  _ __   \ \   / /  \/  |
 \ \ / / _ \| '_ \|  \| |/ _ \ | | | '_ ` _ \ / _` | '_ \| '_ \   \ \ / /| |\/| |
  \ V / (_) | | | | |\  |  __/ |_| | | | | | | (_| | | | | | | |   \ V / | |  | |
   \_/ \___/|_| |_|_| \_|\___|\__,_|_| |_| |_|\__,_|_| |_|_| |_|    \_/  |_|  |_|

<div style="background:black">                                                                     --Ver. 1.0--

Type 'help' for available commands.

~ \$
</div>

Os comandos disponíveis estão apresentados a seguir. São descritos em maior detalhe por meio do comando `help`.

| comando | efeito |
|---------|--------|
| `assemble [src] [-w] [-o out]` | monta um arquivo-fonte em assembly para um arquivo de listagem e um em código-objeto.
| `exit` | sai do programa.
| `help` | imprime a mensagem de ajuda.
| `load [program]` | carrega um programa em código-objeto para a memória da MVN.
| `ls` | lista os arquivos na pasta `/filesystem`.
| `print [file]`| imprime o conteúdo de um arquivo.
| `md [address] [range]` | imprime o conteúdo da memória da MVN em *chunks* de 16 posições seguidas.
| `mm [address] [data]` | modifica o conteúdo da memória da MVN.
| `rm [file(1)]...[file(n)]` | remove um ou mais arquivos da pasta `/filesystem`
| `run [address]` | aciona a execução da MVN. É possível especificar o endereço inicial.
| `set [register] [data]` | modifica o conteúdo de um registrador (AC ou PC).
| `status` | imprime o conteúdo dos registradores da MVN.
| `step [address]` | executa as instruções da MVN passo-a-passo, pressionando `enter`.
| `turn [on off]` | liga ou desliga a MVN. |

### Assembler

O *assembler* da MVN foi escrito em alto-nível. Trata-se de um montador **absoluto**, isto é, todas as posições de memória
já estão definidas. Para utilizá-lo, basta digitar o comando `assemble [src]` na linha de comando do programa, em que 
`[src]` é um arquivo-fonte em *assembly* localizado no diretório `/filesystem`.

A estrutura de um programa típico da MVN é descrita a seguir:

<div style="background:#F0F0F0">
<code style="background:#F0F0F0;color:black">
; Comentários iniciam-se com ';'

        ORG 100  ; Posição da primeira instrução
        
START:  ADD $120  ; Labels terminam com ':'
        SUB 440   ; Operandos podem estar tanto em decimal quanto em hexadecimal
        MUL VAR1  ; Operandos podem ser outras labels também
        GD        ; Tanto GD quanto
        PD        ; PD nâo recebem operandos.
        ...
        JP  START
        
        ORG 120  ; Posição da próxima instrução
        
VAR1:   CON $F2   ; Constantes só possuem um byte de tamanho
        END START ; Fim do programa
        MM  VAR1  ; Não será processado pelo assembler
    
</code>
</div>

Note que, além das instruções de máquina da MVN, existem três pseudo-instruções do assembler:

| mnemônico | efeito |
| --------- | ------ |
| ORG | Modifica o endereço das próximas instruções. |
| CON | Define uma constante de um byte. |
| END | Indica ao *assembler* que o programa acabou. |

Os endereços disponíveis para os programas do usuário iniciam-se na posição `$070` (ver *loader*)

### Loader

O *loader* foi implementado em linguagem de máquina, e ocupa os primeiros 148 bytes da memória da MVN. Ele é injetado pela
interface na inicialização da MVN. Para utilizá-lo, basta digitar o comando `load [program]` na linha de comando do programa,
em que `[program]` é um código-objeto no formato '.vnc'.

O formatao '.vnc' nada mais é do que um arquivo em *plain-text*, em que os dados são escritos em "hexadecimal". Os dados a
serem escritos na memória da MVN são divididos em blocos, cuja estrutura é:

O programa-fonte do *loader* encontra-se a seguir: